In [ ]:
f(x) = cos(x)

# Метод Свенна
function svenn(f, x0, h)
    fm, f0, fp = f(x0 - h), f(x0), f(x0 + h)
    if fm >= f0 >= fp
        dir = 1.0
    elseif fm <= f0 <= fp
        dir = -1.0
    elseif fm >= f0 && fp >= f0
        return (x0 - h, x0 + h)
    else
        dir = fm < fp ? -1.0 : 1.0
    end
    step = h * dir
    x_prev, x_curr = x0, x0 + step
    for _ in 1:50
        step *= 2
        x_next = x_curr + step
        if f(x_next) > f(x_curr)
            return (min(x_prev, x_next), max(x_prev, x_next))
        end
        x_prev, x_curr = x_curr, x_next
    end
    return (min(x_prev, x_curr), max(x_prev, x_curr))
end

# Метод Фибоначчи
function fibonacci(f, a, b, eps)
    F = [1, 1]
    while F[end] < (b - a) / eps
        push!(F, F[end] + F[end-1])
    end
    n = length(F) - 1

    x1 = a + F[n-1] / F[n+1] * (b - a)
    x2 = a + F[n] / F[n+1] * (b - a)
    f1, f2 = f(x1), f(x2)

    for k in 2:n
        if f1 > f2
            a, x1, f1 = x1, x2, f2
            idx = n - k + 1
            x2 = (idx >= 1 && idx + 1 <= length(F)) ? a + F[idx] / F[idx+1] * (b - a) : x1 + eps
            f2 = f(x2)
        else
            b, x2, f2 = x2, x1, f1
            idx = n - k
            x1 = (idx >= 1 && idx + 2 <= length(F)) ? a + F[idx] / F[idx+2] * (b - a) : x2 - eps
            f1 = f(x1)
        end
    end
    return (a + b) / 2, b - a
end

# Численные производные
df(f, x; h=1e-8) = (f(x + h) - f(x - h)) / (2h)
d2f(f, x; h=1e-5) = (f(x + h) - 2f(x) + f(x - h)) / h^2

# Проверка унимодальности
function check_unimodality(f, a, b; n=1000)
    xs = range(a, b, length=n)
    derivs = [df(f, x) for x in xs]
    changes = 0
    for i in 2:length(derivs)
        if derivs[i-1] * derivs[i] < 0
            changes += 1
        end
    end
    return changes
end

# Поиск экстремума
function find_extremum(f, x0, h, eps; mode=:min)
    g = mode == :max ? x -> -f(x) : f
    label = mode == :max ? "МАКСИМУМ" : "МИНИМУМ"
    println("=== $label ===")
    a, b = svenn(g, x0, h)
    println("Свенн: [$a, $b]")
    x_star, prec = fibonacci(g, a, b, eps)
    println("Фибоначчи: x* = $x_star, f(x*) = $(f(x_star)), точность = $prec")
    changes = check_unimodality(g, a, b)
    println("Унимодальность: смен знака f' = $changes → $(changes == 1 ? "унимодальна" : "не унимодальна")")
    fp = df(f, x_star)
    fpp = d2f(f, x_star)
    if mode == :max
        println("Правило дождя: f'(x*) = $(round(fp, digits=10)), f''(x*) = $(round(fpp, digits=6)) → $(fpp < 0 ? "максимум" : "не максимум")")
    else
        println("Правило дождя: f'(x*) = $(round(fp, digits=10)), f''(x*) = $(round(fpp, digits=6)) → $(fpp > 0 ? "минимум" : "не минимум")")
    end
    println()
end

eps = 1e-6
println("f(x) = cos(x), ε = $eps\n")
find_extremum(f, 2.0, 0.5, eps; mode=:min)
find_extremum(f, 5.0, 0.5, eps; mode=:max)


f(x) = cos(x), ε = 1e-6

Свенн: [2.5, 5.5]
Фибоначчи: x* = 3.141592128192368, f(x*) = -0.999999999999862, точность = 8.511657281218277e-7
Унимодальность: смен знака f' = 1 → унимодальна
Правило дождя: f'(x*) = -5.274e-7, f''(x*) = 1.000001 → минимум

|x* - π| = 5.253974251218096e-7
